In [143]:
import pandas as pd
import numpy as np
import math
import re
import time

In [134]:
def combine_categories(row):
    result = row.category_1.lower()
    
    if row.category_2:
        result = result + " " + row.category_2.lower()
    if row.category_3:
        result = result + " " + row.category_3.lower()
    if row.category_4:
        result = result + " " + row.category_4.lower()
    if row.category_5:
        result = result + " " + row.category_5.lower()
    if row.category_6:
        result = result + " " + row.category_6.lower()
    if row.category_7:
        result = result + " " + row.category_7.lower()
    
    result = set(re.split(r"[\>|\&| ]+", result))
    return result

In [119]:
# https://www.google.com/basepages/producttype/taxonomy.en-US.txt
cat = pd.read_csv("./data/taxonomy.csv", encoding="ISO-8859-1")
cat.fillna(False, inplace=True)
cat["combined"] = cat.apply(lambda x: combine_categories(x), axis=1)
print(cat.shape)
cat.head(2)

(5427, 8)


,category_1,category_2,category_3,category_4,category_5,category_6,category_7,combined
0,Animals & Pet Supplies,False,False,False,False,False,False,"{supplies, pet, animals}"
1,Animals & Pet Supplies,Live Animals,False,False,False,False,False,"{supplies, pet, animals, live}"


In [94]:
df = pd.read_csv("./data/emails.csv", encoding="ISO-8859-1")
print(df.shape)
df.head(2)

(500, 2)


,subject,full_text
0,Fine-tune your collection with Edox + Versace,"PLUS, enter our Invicta Big Idea Sweepstakes f..."
1,Des Moines Register Evening Report,Having trouble viewing this email? | View it i...


In [136]:
def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')

In [141]:
def combined_field_mapper(subject, category_words):
    category = set()
        
    for w in category_words:
        if contains_word(subject, w):
            category.add(i)

    return list(category)

In [ ]:
start_time = time.time()

categories = []

for subject in subjects:
    category = set()
    for i, row in cat.iterrows():
        words = row["combined"] # set
        subject = set(subject.split(" ")) # set
        for w in words:
            if contains_word(subject, w):
                category.add(i)
    categories.append(list(category))

# df.categories = categories
print("--- %s seconds ---" % (time.time() - start_time))

In [146]:
# subjects = df.iloc[0:10].subject.apply(lambda x: x.lower())

# start_time = time.time()
# categories = []

# for subject in subjects:
#     category = set()
#     for i, row in cat.iterrows():
#         words = row["combined"]
#         for w in words:
#             if contains_word(subject, w):
#                 category.add(i)
#     categories.append(list(category))

# print("--- %s seconds ---" % (time.time() - start_time))
# --- 11.866845846176147 seconds ---
# # df.categories = categories

In [132]:
# def print_categories(list_of_ids):
#     for i in list_of_ids:
#         category = cat.iloc[i, :-1]
#         print(" > ".join([i for i in category if i != False]))
        
# print_categories(list(category))

Toys & Games > Toys > Riding Toys > Rocking & Spring Riding Toys
Electronics > Audio > Audio Components > Headphones & Headsets
Toys & Games > Toys > Activity Toys > Coiled Spring Toys


In [ ]:
# next steps
# get this to run faster
# clean up categories and assess results
# how do we handle singular v.s. plural?
# should we remove stop words?
# what categories should we add? (shoes example)
# using more than just subject (e.g. full_text)
# using google vision to understand the email better (image recognition, OCR, etc)
### use this info to add new categories
# explore using different weights (would this make sense for us?)